In [1]:
import os
import pandas as pd
import datetime
import re
import logging

# Setup basic logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def list_chat_files(date_directory):
    chat_files = []
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            for team_folder in os.listdir(date_path):
                if team_folder != "KAM":
                    continue
                
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):
                                    chat_files.append(os.path.join(person_path, file))
    logging.debug(f"Chat files listed: {chat_files}")
    return chat_files

def parse_chat_file(file_path, expected_date_minus_one):
    chat_data = []
    last_non_person_time = None  # Tracks the time of the last non-person message
    delay_count = 0  # To count the number of delays

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            message_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*?): (.*)', line)
            system_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*)', line)
            if message_match:
                date_time_str, sender, message = message_match.groups()
            elif system_match:
                date_time_str, info = system_match.groups()
                sender = None
            else:
                continue

            date_time = pd.to_datetime(date_time_str, format='%d/%m/%y, %I:%M %p')

            if date_time.date() != expected_date_minus_one:
                continue

            is_person = sender is not None and re.match(r'^[+\d\s-]+$', sender) is None

            # Calculate delay
            delay = False
            if is_person and last_non_person_time:
                diff = date_time - last_non_person_time
                delay = diff.total_seconds() > 900  # 15 minutes in seconds
                if delay:
                    delay_count += 1

            chat_data.append((date_time, sender, is_person, delay))

            # Update last_non_person_time for non-person messages
            if not is_person:
                last_non_person_time = date_time

    logging.debug(f"File parsed: {file_path}. Delays detected: {delay_count}")
    return chat_data


def create_template_dataframe():
    times = [datetime.datetime(2000, 1, 1, 0, 0) + datetime.timedelta(minutes=1 * i) for i in range(1440)]
    intervals = [time.strftime('%I:%M %p') for time in times]
    df = pd.DataFrame(index=pd.to_datetime(intervals).strftime('%I:%M %p').unique())  # Ensure unique intervals
    return df

def populate_dataframe(df, parsed_data, group_name):
    # Define new column names
    person_col = f"{group_name}_person"
    others_col = f"{group_name}_others"
    delay_col = f"{group_name}_delay"

    # Initialize new columns
    if person_col not in df.columns:
        df[person_col] = 0
    if others_col not in df.columns:
        df[others_col] = 0
    if delay_col not in df.columns:
        df[delay_col] = 0

    # Populate the new columns with parsed data
    for date_time, sender, is_person, delay in parsed_data:
        interval_index = min((date_time.hour * 60 + date_time.minute) // 1, 1439)
        interval = df.index[interval_index]

        if is_person:
            df.at[interval, person_col] = 1
        else:
            df.at[interval, others_col] = 1

        if delay:
            df.at[interval, delay_col] = 1

        logging.debug(f"Updated DataFrame at {interval} for {group_name}: Person={is_person}, Delay={delay}")

    # Update active_chat column
    if 'active_chat' not in df.columns:
        df['active_chat'] = 0

    relevant_columns = [person_col, others_col]
    df['active_chat'] = df[relevant_columns].any(axis=1).astype(int)

    return df

def extract_group_name(file_path):
    group_name = os.path.basename(file_path).replace('WhatsApp Chat with ', '').split('.')[0]
    group_name = re.sub(r'\(\d+\)$', '', group_name)  # Remove any numbers in parentheses at the end
    return group_name  # Removed the extra "_person" suffix

date_directory = "C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Chat Folder from Drive\\drive-download-20231204T064112Z-001"
chat_files = list_chat_files(date_directory)
dataframes = {}

for file in chat_files:
    parts = file.split(os.sep)
    date_folder, person = parts[-4], parts[-2]

    try:
        folder_date = pd.to_datetime(date_folder, format='%Y-%m-%d').date()
    except ValueError:
        continue

    expected_date_minus_one = folder_date - datetime.timedelta(days=1)
    key = f"{folder_date.strftime('%Y-%m-%d')}_{person}"

    # Extract group_name using the dedicated function
    group_name = extract_group_name(file)

    if key not in dataframes:
        dataframes[key] = create_template_dataframe()
    
    # Get parsed_data without expecting group_name in return
    parsed_data = parse_chat_file(file, expected_date_minus_one)
    
    dataframes[key] = populate_dataframe(dataframes[key], parsed_data, group_name)
    logging.debug(f"Dataframe created for key: {key}")

# Example to show a dataframe
example_key = next(iter(dataframes))  # Just for demonstration
logging.debug(f"Example dataframe for key {example_key}: \n{dataframes[example_key]}")


2023-12-04 13:45:44,949 - DEBUG - Chat files listed: ['C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Chat Folder from Drive\\drive-download-20231204T064112Z-001\\2023-12-03\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Agape 2K23NOV2370R.txt', 'C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Chat Folder from Drive\\drive-download-20231204T064112Z-001\\2023-12-03\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Angeline 2K23FEB2804.txt', 'C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Chat Folder from Drive\\drive-download-20231204T064112Z-001\\2023-12-03\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Auxillia 2K23JUL1004.txt', 'C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Chat Folder from Drive\\drive-download-20231204T064112Z-001\\2023-12-03\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Chiedza 2K22FEB1213.txt', 'C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Chat Folder from Drive\\drive-download-20231204T064112Z-001\\2023-12-03\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Courage 2K22JUL1682.txt', 'C:\\Users\\mauriceyeng\\

C:\Users\mauriceyeng\AppData\Local\Temp\ipykernel_12324\1192516396.py:75: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.DataFrame(index=pd.to_datetime(intervals).strftime('%I:%M %p').unique())  # Ensure unique intervals
2023-12-04 13:45:45,076 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Ashi_Edoofa\WhatsApp Chat with Agape 2K23NOV2370R.txt. Delays detected: 1
2023-12-04 13:45:45,078 - DEBUG - Updated DataFrame at 01:14 AM for Agape 2K23NOV2370R: Person=False, Delay=False
2023-12-04 13:45:45,078 - DEBUG - Updated DataFrame at 07:08 AM for Agape 2K23NOV2370R: Person=False, Delay=False
2023-12-04 13:45:45,079 - DEBUG - Updated DataFrame at 07:08 AM for Agape 2K23NOV2370R: Person=False, Delay=False
2023-12-04 13:45:45,079 - DEBUG - Updated DataFrame a

2023-12-04 13:45:45,222 - DEBUG - Updated DataFrame at 11:54 PM for Auxillia 2K23JUL1004: Person=False, Delay=False
2023-12-04 13:45:45,223 - DEBUG - Dataframe created for key: 2023-12-03_Ashi_Edoofa
2023-12-04 13:45:45,266 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Ashi_Edoofa\WhatsApp Chat with Chiedza 2K22FEB1213.txt. Delays detected: 0
2023-12-04 13:45:45,268 - DEBUG - Updated DataFrame at 08:34 AM for Chiedza 2K22FEB1213: Person=False, Delay=False
2023-12-04 13:45:45,269 - DEBUG - Dataframe created for key: 2023-12-03_Ashi_Edoofa
2023-12-04 13:45:45,302 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Ashi_Edoofa\WhatsApp Chat with Courage 2K22JUL1682.txt. Delays detected: 0
2023-12-04 13:45:45,303 - DEBUG - Updated DataFrame at 08:35 AM for Courage 2K22JUL1682: Person=False, Delay=False
2023-12-04

2023-12-04 13:45:45,385 - DEBUG - Updated DataFrame at 08:42 AM for Danai 2K23SEP1134R: Person=True, Delay=False
2023-12-04 13:45:45,385 - DEBUG - Updated DataFrame at 08:43 AM for Danai 2K23SEP1134R: Person=True, Delay=False
2023-12-04 13:45:45,386 - DEBUG - Updated DataFrame at 08:43 AM for Danai 2K23SEP1134R: Person=True, Delay=False
2023-12-04 13:45:45,386 - DEBUG - Updated DataFrame at 08:43 AM for Danai 2K23SEP1134R: Person=True, Delay=False
2023-12-04 13:45:45,386 - DEBUG - Updated DataFrame at 08:44 AM for Danai 2K23SEP1134R: Person=True, Delay=False
2023-12-04 13:45:45,387 - DEBUG - Updated DataFrame at 08:54 AM for Danai 2K23SEP1134R: Person=False, Delay=False
2023-12-04 13:45:45,388 - DEBUG - Updated DataFrame at 08:58 AM for Danai 2K23SEP1134R: Person=True, Delay=False
2023-12-04 13:45:45,388 - DEBUG - Updated DataFrame at 08:58 AM for Danai 2K23SEP1134R: Person=True, Delay=False
2023-12-04 13:45:45,388 - DEBUG - Updated DataFrame at 08:58 AM for Danai 2K23SEP1134R: Person=

2023-12-04 13:45:45,627 - DEBUG - Updated DataFrame at 09:59 AM for Kudakwashe 2K23NOV2061R: Person=True, Delay=False
2023-12-04 13:45:45,628 - DEBUG - Updated DataFrame at 10:00 AM for Kudakwashe 2K23NOV2061R: Person=False, Delay=False
2023-12-04 13:45:45,628 - DEBUG - Updated DataFrame at 10:02 AM for Kudakwashe 2K23NOV2061R: Person=False, Delay=False
2023-12-04 13:45:45,629 - DEBUG - Updated DataFrame at 10:02 AM for Kudakwashe 2K23NOV2061R: Person=True, Delay=False
2023-12-04 13:45:45,630 - DEBUG - Dataframe created for key: 2023-12-03_Ashi_Edoofa
2023-12-04 13:45:45,656 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Ashi_Edoofa\WhatsApp Chat with Kupakwashe 2K23NOV1024.txt. Delays detected: 0
2023-12-04 13:45:45,657 - DEBUG - Updated DataFrame at 07:41 AM for Kupakwashe 2K23NOV1024: Person=True, Delay=False
2023-12-04 13:45:45,658 - DEBUG - Updated DataFrame at 09:30 AM for Kupakwashe 2K23N

...

2023-12-04 13:45:48,063 - DEBUG - Updated DataFrame at 09:04 AM for Tinotenda 2K23AUG1863: Person=False, Delay=False
2023-12-04 13:45:48,064 - DEBUG - Updated DataFrame at 09:10 AM for Tinotenda 2K23AUG1863: Person=True, Delay=False
2023-12-04 13:45:48,064 - DEBUG - Updated DataFrame at 09:13 AM for Tinotenda 2K23AUG1863: Person=False, Delay=False
2023-12-04 13:45:48,065 - DEBUG - Updated DataFrame at 09:19 AM for Tinotenda 2K23AUG1863: Person=True, Delay=False
2023-12-04 13:45:48,065 - DEBUG - Updated DataFrame at 09:31 AM for Tinotenda 2K23AUG1863: Person=False, Delay=False
2023-12-04 13:45:48,066 - DEBUG - Updated DataFrame at 10:50 AM for Tinotenda 2K23AUG1863: Person=True, Delay=True
2023-12-04 13:45:48,066 - DEBUG - Updated DataFrame at 10:52 AM for Tinotenda 2K23AUG1863: Person=False, Delay=False
2023-12-04 13:45:48,067 - DEBUG - Updated DataFrame at 10:53 AM for Tinotenda 2K23AUG1863: Person=True, Delay=False
2023-12-04 13:45:48,067 - DEBUG - Updated DataFrame at 10:55 AM for T

2023-12-04 13:45:48,457 - DEBUG - Updated DataFrame at 11:19 AM for Nyasha 2K22JUL2255: Person=False, Delay=False
2023-12-04 13:45:48,459 - DEBUG - Dataframe created for key: 2023-12-03_Milan_Edoofa
2023-12-04 13:45:48,477 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Milan_Edoofa\WhatsApp Chat with Petronela 2K22JUN0950.txt. Delays detected: 0
2023-12-04 13:45:48,479 - DEBUG - Updated DataFrame at 12:35 PM for Petronela 2K22JUN0950: Person=False, Delay=False
2023-12-04 13:45:48,480 - DEBUG - Dataframe created for key: 2023-12-03_Milan_Edoofa
2023-12-04 13:45:48,519 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Milan_Edoofa\WhatsApp Chat with Shantel 2K23JAN0755.txt. Delays detected: 0
2023-12-04 13:45:48,520 - DEBUG - Updated DataFrame at 08:34 AM for Shantel 2K23JAN0755: Person=False, Delay=False
2023

2023-12-04 13:45:48,839 - DEBUG - Updated DataFrame at 08:26 AM for Esther 2K23NOV1792: Person=True, Delay=False
2023-12-04 13:45:48,840 - DEBUG - Updated DataFrame at 09:54 AM for Esther 2K23NOV1792: Person=False, Delay=False
2023-12-04 13:45:48,840 - DEBUG - Updated DataFrame at 10:10 AM for Esther 2K23NOV1792: Person=True, Delay=True
2023-12-04 13:45:48,841 - DEBUG - Updated DataFrame at 11:03 AM for Esther 2K23NOV1792: Person=False, Delay=False
2023-12-04 13:45:48,841 - DEBUG - Updated DataFrame at 11:03 AM for Esther 2K23NOV1792: Person=False, Delay=False
2023-12-04 13:45:48,842 - DEBUG - Updated DataFrame at 01:41 PM for Esther 2K23NOV1792: Person=True, Delay=True
2023-12-04 13:45:48,842 - DEBUG - Updated DataFrame at 02:01 PM for Esther 2K23NOV1792: Person=False, Delay=False
2023-12-04 13:45:48,844 - DEBUG - Updated DataFrame at 08:34 PM for Esther 2K23NOV1792: Person=False, Delay=False
2023-12-04 13:45:48,845 - DEBUG - Dataframe created for key: 2023-12-03_Shivjeet Edoofa
2023-

2023-12-04 13:45:49,066 - DEBUG - Updated DataFrame at 10:20 AM for Pearl 2K23NOV2309: Person=True, Delay=False
2023-12-04 13:45:49,067 - DEBUG - Updated DataFrame at 10:26 AM for Pearl 2K23NOV2309: Person=False, Delay=False
2023-12-04 13:45:49,068 - DEBUG - Updated DataFrame at 10:37 AM for Pearl 2K23NOV2309: Person=True, Delay=False
2023-12-04 13:45:49,068 - DEBUG - Updated DataFrame at 12:20 PM for Pearl 2K23NOV2309: Person=False, Delay=False
2023-12-04 13:45:49,069 - DEBUG - Updated DataFrame at 01:34 PM for Pearl 2K23NOV2309: Person=True, Delay=True
2023-12-04 13:45:49,069 - DEBUG - Updated DataFrame at 08:28 PM for Pearl 2K23NOV2309: Person=False, Delay=False
2023-12-04 13:45:49,071 - DEBUG - Dataframe created for key: 2023-12-03_Shivjeet Edoofa
2023-12-04 13:45:49,095 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Shivjeet Edoofa\WhatsApp Chat with Praise 2K23JAN0535.txt. Delays detected:

2023-12-04 13:45:49,392 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Shivjeet Edoofa\WhatsApp Chat with Tawanda 2K23OCT1754.txt. Delays detected: 1
2023-12-04 13:45:49,394 - DEBUG - Updated DataFrame at 05:33 AM for Tawanda 2K23OCT1754: Person=False, Delay=False
2023-12-04 13:45:49,395 - DEBUG - Updated DataFrame at 08:20 AM for Tawanda 2K23OCT1754: Person=True, Delay=True
2023-12-04 13:45:49,395 - DEBUG - Updated DataFrame at 10:37 AM for Tawanda 2K23OCT1754: Person=False, Delay=False
2023-12-04 13:45:49,396 - DEBUG - Updated DataFrame at 10:40 AM for Tawanda 2K23OCT1754: Person=True, Delay=False
2023-12-04 13:45:49,397 - DEBUG - Updated DataFrame at 10:42 AM for Tawanda 2K23OCT1754: Person=False, Delay=False
2023-12-04 13:45:49,397 - DEBUG - Updated DataFrame at 10:42 AM for Tawanda 2K23OCT1754: Person=False, Delay=False
2023-12-04 13:45:49,398 - DEBUG - Updated DataFrame at 10:43 AM for Taw

In [3]:
# Directory to save CSV files
csv_save_directory = "C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Chat CSVs"
os.makedirs(csv_save_directory, exist_ok=True)

# Saving each DataFrame as a CSV
for key, df in dataframes.items():
    csv_file_path = os.path.join(csv_save_directory, f"{key}.csv")
    df.to_csv(csv_file_path)
    print(f"file saved as {key}")

file saved as 2023-12-03_Ashi_Edoofa
file saved as 2023-12-03_Kirti Edoofa
file saved as 2023-12-03_Milan_Edoofa
file saved as 2023-12-03_Shivjeet Edoofa


# Fixing bugs, hehe, still good news

In [5]:
# Directory to save CSV files
csv_save_directory = "C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Chat CSVs"
os.makedirs(csv_save_directory, exist_ok=True)

# Saving each DataFrame as a CSV
for key, df in dataframes.items():
    csv_file_path = os.path.join(csv_save_directory, f"{key}_v2.csv")
    df.to_csv(csv_file_path)
    print(f"file saved as {key}_v2")

file saved as 2023-12-03_Ashi_Edoofa_v2
file saved as 2023-12-03_Kirti Edoofa_v2
file saved as 2023-12-03_Milan_Edoofa_v2
file saved as 2023-12-03_Shivjeet Edoofa_v2


In [8]:
import os
import pandas as pd
import datetime
import re
import logging

# Setup basic logging
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

def list_chat_files(date_directory):
    chat_files = []
    for date_folder in os.listdir(date_directory):
        date_path = os.path.join(date_directory, date_folder)
        if os.path.isdir(date_path):
            for team_folder in os.listdir(date_path):
                if team_folder != "KAM":
                    continue
                
                team_path = os.path.join(date_path, team_folder)
                if os.path.isdir(team_path):
                    for person_folder in os.listdir(team_path):
                        person_path = os.path.join(team_path, person_folder)
                        if os.path.isdir(person_path):
                            for file in os.listdir(person_path):
                                if file.endswith('.txt'):
                                    chat_files.append(os.path.join(person_path, file))
    logging.debug(f"Chat files listed: {chat_files}")
    return chat_files

def parse_chat_file(file_path, expected_date_minus_one):
    chat_data = []
    last_non_person_time = None  # Tracks the time of the last non-person message
    delay_count = 0  # To count the number of delays

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            message_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*?): (.*)', line)
            system_match = re.match(r'(\d{2}/\d{2}/\d{2}, \d{1,2}:\d{2} [ap]m) - (.*)', line)
            if message_match:
                date_time_str, sender, message = message_match.groups()
            elif system_match:
                date_time_str, info = system_match.groups()
                sender = None
            else:
                continue

            date_time = pd.to_datetime(date_time_str, format='%d/%m/%y, %I:%M %p')

            if date_time.date() != expected_date_minus_one:
                continue

            is_person = sender is not None and re.match(r'^[+\d\s-]+$', sender) is None

            # Calculate delay
            delay = False
            if is_person and last_non_person_time:
                diff = date_time - last_non_person_time
                delay = diff.total_seconds() > 900  # 15 minutes in seconds
                if delay:
                    delay_count += 1

            chat_data.append((date_time, sender, is_person, delay))

            # Update last_non_person_time for non-person messages
            if not is_person:
                last_non_person_time = date_time

    logging.debug(f"File parsed: {file_path}. Delays detected: {delay_count}")
    return chat_data

def create_template_dataframe():
    times = [datetime.datetime(2000, 1, 1, 0, 0) + datetime.timedelta(minutes=1 * i) for i in range(1440)]
    intervals = [time.strftime('%I:%M %p') for time in times]
    df = pd.DataFrame(index=pd.to_datetime(intervals).strftime('%I:%M %p').unique())  # Ensure unique intervals
    return df

def populate_dataframe(df, parsed_data, group_name):
    # Define new column names
    person_col = f"{group_name}_person"
    others_col = f"{group_name}_others"
    delay_col = f"{group_name}_delay"

    # Initialize new columns
    if person_col not in df.columns:
        df[person_col] = 0
    if others_col not in df.columns:
        df[others_col] = 0
    if delay_col not in df.columns:
        df[delay_col] = 0

    # Populate the new columns with parsed data
    for date_time, sender, is_person, delay in parsed_data:
        interval_index = min((date_time.hour * 60 + date_time.minute) // 1, 1439)
        interval = df.index[interval_index]

        if is_person:
            df.at[interval, person_col] = 1
        else:
            df.at[interval, others_col] = 1

        if delay:
            df.at[interval, delay_col] = 1

        logging.debug(f"Updated DataFrame at {interval} for {group_name}: Person={is_person}, Delay={delay}")

    return df

def calculate_active_chats(df):
    active_chats_df = create_template_dataframe()
    active_chats_df['active_chats'] = 0

    person_cols = [col for col in df.columns if col.endswith('_person')]
    others_cols = [col for col in df.columns if col.endswith('_others')]

    for index, row in df.iterrows():
        # Check if there are any person or others columns
        if person_cols and others_cols:
            person_col = person_cols[0]
            others_col = others_cols[0]

            if row[person_col] > 0 or row[others_col] > 0:
                active_chats_df.at[index, 'active_chats'] = 1

    return active_chats_df



def extract_group_name(file_path):
    group_name = os.path.basename(file_path).replace('WhatsApp Chat with ', '').split('.')[0]
    group_name = re.sub(r'\(\d+\)$', '', group_name)  # Remove any numbers in parentheses at the end
    return group_name  # Removed the extra "_person" suffix

date_directory = "C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Chat Folder from Drive\\drive-download-20231204T064112Z-001"
chat_files = list_chat_files(date_directory)
dataframes = {}

for file in chat_files:
    parts = file.split(os.sep)
    date_folder, person = parts[-4], parts[-2]

    try:
        folder_date = pd.to_datetime(date_folder, format='%Y-%m-%d').date()
    except ValueError:
        continue

    expected_date_minus_one = folder_date - datetime.timedelta(days=1)
    key = f"{folder_date.strftime('%Y-%m-%d')}_{person}"

    # Extract group_name using the dedicated function
    group_name = extract_group_name(file)

    if key not in dataframes:
        dataframes[key] = create_template_dataframe()
    
    # Get parsed_data without expecting group_name in return
    parsed_data = parse_chat_file(file, expected_date_minus_one)
    
    # Calculate active chats for each DataFrame
    active_chats_df = calculate_active_chats(dataframes[key])

    # Merge active_chats_df with main DataFrame
    merged_df = pd.concat([dataframes[key].iloc[:, :1], active_chats_df, dataframes[key].iloc[:, 1:]], axis=1)
    dataframes[key] = merged_df

    logging.debug(f"Dataframe created for key: {key}")

# Example to show a dataframe
example_key = next(iter(dataframes))  # Just for demonstration
logging.debug(f"Example dataframe for key {example_key}: \n{dataframes[example_key]}")


2023-12-04 15:27:52,040 - DEBUG - Chat files listed: ['C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Chat Folder from Drive\\drive-download-20231204T064112Z-001\\2023-12-03\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Agape 2K23NOV2370R.txt', 'C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Chat Folder from Drive\\drive-download-20231204T064112Z-001\\2023-12-03\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Angeline 2K23FEB2804.txt', 'C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Chat Folder from Drive\\drive-download-20231204T064112Z-001\\2023-12-03\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Auxillia 2K23JUL1004.txt', 'C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Chat Folder from Drive\\drive-download-20231204T064112Z-001\\2023-12-03\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Chiedza 2K22FEB1213.txt', 'C:\\Users\\mauriceyeng\\Python\\Daily-Reports\\Chat Folder from Drive\\drive-download-20231204T064112Z-001\\2023-12-03\\KAM\\Ashi_Edoofa\\WhatsApp Chat with Courage 2K22JUL1682.txt', 'C:\\Users\\mauriceyeng\\

C:\Users\mauriceyeng\AppData\Local\Temp\ipykernel_12324\2979815787.py:74: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.DataFrame(index=pd.to_datetime(intervals).strftime('%I:%M %p').unique())  # Ensure unique intervals
2023-12-04 15:27:52,152 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Ashi_Edoofa\WhatsApp Chat with Agape 2K23NOV2370R.txt. Delays detected: 1
C:\Users\mauriceyeng\AppData\Local\Temp\ipykernel_12324\2979815787.py:74: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.DataFrame(index=pd.to_datetime(intervals).strftime('%I:%M %p').unique())  # Ensure unique intervals
2023-12-04 15:27:52,304 - D

C:\Users\mauriceyeng\AppData\Local\Temp\ipykernel_12324\2979815787.py:74: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.DataFrame(index=pd.to_datetime(intervals).strftime('%I:%M %p').unique())  # Ensure unique intervals
2023-12-04 15:27:54,333 - DEBUG - Dataframe created for key: 2023-12-03_Ashi_Edoofa
2023-12-04 15:27:54,352 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Ashi_Edoofa\WhatsApp Chat with Kudakwashe 2K23NOV2061R.txt. Delays detected: 0
C:\Users\mauriceyeng\AppData\Local\Temp\ipykernel_12324\2979815787.py:74: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.DataFrame(index=pd.to_datetime(interva

C:\Users\mauriceyeng\AppData\Local\Temp\ipykernel_12324\2979815787.py:74: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.DataFrame(index=pd.to_datetime(intervals).strftime('%I:%M %p').unique())  # Ensure unique intervals
2023-12-04 15:27:56,476 - DEBUG - Dataframe created for key: 2023-12-03_Ashi_Edoofa
2023-12-04 15:27:56,514 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Ashi_Edoofa\WhatsApp Chat with Sharmaine 2K23OCT0695.txt. Delays detected: 4
C:\Users\mauriceyeng\AppData\Local\Temp\ipykernel_12324\2979815787.py:74: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.DataFrame(index=pd.to_datetime(intervals

2023-12-04 15:27:58,552 - DEBUG - Dataframe created for key: 2023-12-03_Kirti Edoofa
2023-12-04 15:27:58,613 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Kirti Edoofa\WhatsApp Chat with Anotidaishe 2K23JUN0971.txt. Delays detected: 0
C:\Users\mauriceyeng\AppData\Local\Temp\ipykernel_12324\2979815787.py:74: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.DataFrame(index=pd.to_datetime(intervals).strftime('%I:%M %p').unique())  # Ensure unique intervals
2023-12-04 15:27:58,797 - DEBUG - Dataframe created for key: 2023-12-03_Kirti Edoofa
2023-12-04 15:27:58,834 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Kirti Edoofa\WhatsApp Chat with Blessing 2K23JUN2327.txt

2023-12-04 15:28:00,848 - DEBUG - Dataframe created for key: 2023-12-03_Kirti Edoofa
2023-12-04 15:28:00,889 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Kirti Edoofa\WhatsApp Chat with Isaac 2K23OCT1366.txt. Delays detected: 0
C:\Users\mauriceyeng\AppData\Local\Temp\ipykernel_12324\2979815787.py:74: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.DataFrame(index=pd.to_datetime(intervals).strftime('%I:%M %p').unique())  # Ensure unique intervals
2023-12-04 15:28:01,036 - DEBUG - Dataframe created for key: 2023-12-03_Kirti Edoofa
2023-12-04 15:28:01,077 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Kirti Edoofa\WhatsApp Chat with Joyce 2K23JUL3169R.txt. Delays

2023-12-04 15:28:03,028 - DEBUG - Dataframe created for key: 2023-12-03_Kirti Edoofa
2023-12-04 15:28:03,065 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Kirti Edoofa\WhatsApp Chat with Shane 2K23OCT2588.txt. Delays detected: 0
C:\Users\mauriceyeng\AppData\Local\Temp\ipykernel_12324\2979815787.py:74: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.DataFrame(index=pd.to_datetime(intervals).strftime('%I:%M %p').unique())  # Ensure unique intervals
2023-12-04 15:28:03,210 - DEBUG - Dataframe created for key: 2023-12-03_Kirti Edoofa
2023-12-04 15:28:03,249 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Kirti Edoofa\WhatsApp Chat with Staylodge 2K23OCT2788.txt. Del

C:\Users\mauriceyeng\AppData\Local\Temp\ipykernel_12324\2979815787.py:74: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.DataFrame(index=pd.to_datetime(intervals).strftime('%I:%M %p').unique())  # Ensure unique intervals
2023-12-04 15:28:05,321 - DEBUG - Dataframe created for key: 2023-12-03_Milan_Edoofa
2023-12-04 15:28:05,353 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Milan_Edoofa\WhatsApp Chat with Mondliwethu 2K23MAY2381.txt. Delays detected: 0
C:\Users\mauriceyeng\AppData\Local\Temp\ipykernel_12324\2979815787.py:74: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.DataFrame(index=pd.to_datetime(inter

2023-12-04 15:28:07,309 - DEBUG - Dataframe created for key: 2023-12-03_Shivjeet Edoofa
2023-12-04 15:28:07,318 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Shivjeet Edoofa\WhatsApp Chat with Chelsie 2K23NOV1864R.txt. Delays detected: 3
C:\Users\mauriceyeng\AppData\Local\Temp\ipykernel_12324\2979815787.py:74: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.DataFrame(index=pd.to_datetime(intervals).strftime('%I:%M %p').unique())  # Ensure unique intervals
2023-12-04 15:28:07,468 - DEBUG - Dataframe created for key: 2023-12-03_Shivjeet Edoofa
2023-12-04 15:28:07,480 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Shivjeet Edoofa\WhatsApp Chat with Esther 2K23NOV1

2023-12-04 15:28:09,290 - DEBUG - Dataframe created for key: 2023-12-03_Shivjeet Edoofa
2023-12-04 15:28:09,343 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Shivjeet Edoofa\WhatsApp Chat with Shepherd 2K23AUG1035.txt. Delays detected: 0
C:\Users\mauriceyeng\AppData\Local\Temp\ipykernel_12324\2979815787.py:74: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.DataFrame(index=pd.to_datetime(intervals).strftime('%I:%M %p').unique())  # Ensure unique intervals
2023-12-04 15:28:09,488 - DEBUG - Dataframe created for key: 2023-12-03_Shivjeet Edoofa
2023-12-04 15:28:09,495 - DEBUG - File parsed: C:\Users\mauriceyeng\Python\Daily-Reports\Chat Folder from Drive\drive-download-20231204T064112Z-001\2023-12-03\KAM\Shivjeet Edoofa\WhatsApp Chat with Solomon 2K23NOV